In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from IPython.display import clear_output
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, TimeSeriesSplit

In [2]:
def find_nan_features(df):
    null_cols = []
    for col in df.columns:
        if df[col].isnull().values.any():
            null_cols.append(col)
    return null_cols

In [3]:
def remove_nan_rows(df):
    # getting indices (rows) of all NaN values
    inds = pd.isnull(df).any(1).nonzero()[0]

    # drop all the rows with NaN values
    return df.drop(df.index[inds])

In [4]:
def split(df, train_fraction):
    mindate = df.Date.min()
    maxdate = df.Date.max()
    splitdate = mindate + (maxdate - mindate) * train_fraction
    train = df[df.Date < splitdate]
    test = df[df.Date >= splitdate]
    return train, test

In [5]:
def get_x_y(df):
    # split set in data and target
    X = df.drop('NumberOfSales', axis=1)
    y = df["NumberOfSales"]
    return X, y

In [6]:
def train_model(X_train, y_train, n_estimators=250, n_jobs=1):
    # fit random forest with 250 trees
    forest = RandomForestRegressor(n_estimators=n_estimators, random_state=0, n_jobs=n_jobs)
    forest.fit(X_train, y_train)
    return forest

In [7]:
def bipmeter(X_val, y_val, y_pred, rows_region, rows_store):
    X_val['NumberOfSales'] = y_val
    X_val['NumberOfSales_predicted'] = y_pred
    X_val['Region'] = rows_region
    X_val['StoreID'] = rows_store
    
    df1 = X_val
    # group by month & region 
    result = df1.groupby(['Month','Region','StoreID'], as_index=False).agg({"NumberOfSales": "sum","NumberOfSales_predicted":"sum"})
    result['Err'] = abs(result['NumberOfSales']-result['NumberOfSales_predicted'])
    #result = result.groupby(['StoreID','Region'], as_index=False).agg({'Err':'sum'})
    result = result.groupby('Region', as_index=False).agg({'Err':'sum'})
    
    #den = df1.groupby(['Month','Region','StoreID'], as_index=False).agg({"NumberOfSales": "sum"})
    #den = den.groupby(['StoreID','Region'], as_index=False).agg({'NumberOfSales':'sum'})
    den = df1.groupby('Region', as_index=False).agg({'NumberOfSales':'sum'})
    
    
    E_r = (result['Err']/den['NumberOfSales'])
    # % error
    return E_r.sum()/len(E_r.tolist())

In [8]:
def bip_metric(X_val, y_val, y_pred, rows_region, rows_store):

    e_r = []
    month_sum = []
    # adjust shape
    X_val['Region'] = rows_region
    X_val = pd.get_dummies(X_val, columns=['Region'], prefix='Region')
èò
    y_pred = y_pred.tolist()
    y_val = y_val.tolist()

    for r in range(11):

        region = 'Region_' + str(r)
        d = X_val.loc[X_val[region] == 1]

        error = 0
        y_somma = 0
  
#  cycle through stores
        for i in range(1000,1736): 
               
            for m in range(1,13):
                sum_pred_month = 0
                sum_actual_month = 0
                indexes = d.index[(d['StoreID'] == i) & (d['Month'] == m)].tolist()
                
                for j in indexes:
           
                    sum_pred_month += y_pred[j]
                    sum_actual_month += y_val[j]

                error += abs(sum_actual_month - sum_pred_month)
                y_somma = y_somma + sum_actual_month

    e_r.append(error/y_somma)

    return sum(e_r)/len(e_r)

In [9]:
def eval_model(X_val, y_val, model, store_ids, val_date, rows_region, rows_store):
    y_pred = model.predict(X_val)
    new_x_val = X_val 
    new_x_val['Month'] = pd.DatetimeIndex(val_date['Date']).month
    new_x_val['StoreID'] = store_ids
    #score = bip_metric(X_val, y_val, y_pred, rows_region, rows_store)
    
    score = bip_metric(X_val, y_val, y_pred, rows_region, rows_store)
    print("bipmeter:  {:6.4f}".format(bipmeter(X_val,y_val,y_pred,rows_region,rows_store)))
    
    return score

In [10]:
def crossvalidation(df,rows_store, rows_region, nfolds=8, n_estimators=50, n_jobs=1, verbose=False):
    '''Crossvalidation on the dataset `df` with `nfolds` folds.
    Split the dataset in N training-validation folds,
    trains and evaluates results for each of them,
    returns the mean of the error and metrics'''
    
    # convert date to datetime
    if df['Date'].dtype == np.dtype('O'):
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
        
    # add temporary columns to compute splits
    df['Year_CV'] = df['Date'].dt.year
    df['Month_CV'] = df['Date'].dt.month
    
    # get all year-month pairs
    l = sorted(list(set(df[['Year_CV', 'Month_CV']].itertuples(index=False))))
    
    # drop temporary columns
    df = df.drop(['Year_CV', 'Month_CV'], axis=1)
    
    # split dataset in folds
    kf = KFold(nfolds)
    indexes = []
    for train_months_idx, val_months_idx in kf.split(l):
        train_months = [l[i] for i in train_months_idx]
        val_months = [l[i] for i in val_months_idx]
        train_indexes = df.Date.apply(lambda d: (d.year, d.month) in train_months)
        val_indexes = df.Date.apply(lambda d: (d.year, d.month) in val_months)
        indexes.append((train_indexes, val_indexes))

    # iterate on the folds
    total_score = 0
    for train_indexes, val_indexes in indexes:
        # split
        df_train = df[train_indexes]
        df_validation = df[val_indexes]
        
        # store Date and ID
        val_date = pd.DataFrame(df_validation['Date'])
        val_store_id = df_validation['StoreID']
        
        # and drop them
        df_train = df_train.drop('Date', axis=1)
        df_validation = df_validation.drop('Date', axis=1)

        df_train = df_train.drop('StoreID', axis=1)
        df_validation = df_validation.drop('StoreID', axis=1)
        
        # train model
        X_train, y_train = get_x_y(df_train)
        model = train_model(X_train, y_train, n_estimators=n_estimators, n_jobs=n_jobs)
        
        # evaluate model
        X_val, y_val = get_x_y(df_validation)
#         months = pd.DatetimeIndex(val_date['Date']).month
        score = eval_model(X_val, y_val, model, val_store_id, val_date, rows_region, rows_store)
        if verbose:
            print('Partial score: {:6.4f}'.format(score))
        total_score += score
        
    return total_score / nfolds

In [11]:
def crossvalidation_TS(df,rows_store, rows_region, nfolds=8, n_estimators=50, n_jobs=1, verbose=False):
    '''Time-Series Crossvalidation on the dataset `df` with `nfolds` folds.
    Split the dataset in N training-validation folds,
    trains and evaluates results for each of them,
    returns the mean of the error and metrics'''
    
    # convert date to datetime
    if df['Date'].dtype == np.dtype('O'):
        df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
        
    # add temporary columns to compute splits
    df['Year_CV'] = df['Date'].dt.year
    df['Month_CV'] = df['Date'].dt.month
    
    # get all year-month pairs
    l = sorted(list(set(df[['Year_CV', 'Month_CV']].itertuples(index=False))))
    
    # drop temporary columns
    df = df.drop(['Year_CV', 'Month_CV'], axis=1)
    
    # split dataset in folds
    kf = TimeSeriesSplit(nfolds)
    indexes = []
    for train_months_idx, val_months_idx in kf.split(l):
        train_months = [l[i] for i in train_months_idx]
        val_months = [l[i] for i in val_months_idx]
        train_indexes = df.Date.apply(lambda d: (d.year, d.month) in train_months)
        val_indexes = df.Date.apply(lambda d: (d.year, d.month) in val_months)
        indexes.append((train_indexes, val_indexes))

    # iterate on the folds
    total_score = 0
    for train_indexes, val_indexes in indexes:
        # split
        df_train = df[train_indexes]
        df_validation = df[val_indexes]
        
        # store Date and ID
        val_date = pd.DataFrame(df_validation['Date'])
        val_store_id = df_validation['StoreID']
        
        # and drop them
        df_train = df_train.drop('Date', axis=1)
        df_validation = df_validation.drop('Date', axis=1)

        df_train = df_train.drop('StoreID', axis=1)
        df_validation = df_validation.drop('StoreID', axis=1)
        
        # train model
        X_train, y_train = get_x_y(df_train)
        model = train_model(X_train, y_train, n_estimators=n_estimators, n_jobs=n_jobs)
        
        # evaluate model
        X_val, y_val = get_x_y(df_validation)
        months = pd.DatetimeIndex(val_date['Date']).month
        score = eval_model(X_val, y_val, model, val_store_id, val_date, rows_region,rows_store)
        if verbose:
            print('Partial score: {:6.4f}'.format(score))
        total_score += score
        
    return total_score / nfolds

### Load dataset

In [12]:
# load preprocessed csv to dataframe
df = pd.read_csv('preprocessed_train_with_avg.csv')

### Prepare dataset

In [13]:
# Save region for each index
rows_region = df['Region']
rows_store = df['StoreID']

# Choose features
selected_features=[
    'NumberOfSales',
    'HasPromotions', 
    'daily_sales',
    'month_avg_sales',
    'yearly_sales',
    'IsOpen_yesterday',
    'DayOfWeek',
    'NearestCompetitor',
    'Week',
    'StoreID',
    'IsHoliday_tomorrow',
    'Date'] # droppata dopo

df = df[selected_features]
# df_train = df[selected_features]
# df_validation = df[selected_features]

In [14]:
# Look for features with NaN values
null_cols = find_nan_features(df)
print('Features with NaN:')
for col in null_cols:
    print(col)
    
# drop all rows with NaN values
df = remove_nan_rows(df)

Features with NaN:
month_avg_sales
IsOpen_yesterday
IsHoliday_tomorrow


### Cross-validation

In [15]:
score = crossvalidation(df,rows_store, rows_region, nfolds=8, 
                        n_estimators=50, n_jobs=3, verbose=True)

print("Total score: {:6.4f}".format(score))

bipmeter:  0.0577
Partial score: 0.0528
bipmeter:  0.0607
Partial score: 0.0600
bipmeter:  0.0449
Partial score: 0.0335
bipmeter:  0.0530
Partial score: 0.0425
bipmeter:  0.0592
Partial score: 0.0484


KeyError: 'Region_2'

In [ ]:
score_ts = crossvalidation_TS(df,rows_store, rows_region, nfolds=8, 
                              n_estimators=50, n_jobs=3, verbose=True)

print("Total score: {:6.4f}".format(score_ts))

#### Cross-validation: Random Forest, 50 trees, 8 folds
    Partial score: 0.0215
    Partial score: 0.0203
    Partial score: 0.0356
    Partial score: 0.0227
    Partial score: 0.0263
    Partial score: 0.0204
    Partial score: 0.0243
    Partial score: 0.0316
    Total score: 0.0253
    
    
#### Bipmeter
    Partial score: 0.0580
    Partial score: 0.0607
    Partial score: 0.0448
    Partial score: 0.0530
    Partial score: 0.0593
    Partial score: 0.0556
    Partial score: 0.0551
    Partial score: 0.0571
    Total score: 0.0554


In [ ]:
## questo notebook:
# senza StoreID
# con NumberOfSales reali
# con tutti i dati

## da fare:
# per store
# con i dati dello store

In [ ]:
# TODO add MAE, MSE, errore per regione